# SDG 7: Rural Electricity Access in Sub-Saharan Africa
## Exploratory Data Analysis Report

---

**Date:** November 2025  
**Scope:** 50 Sub-Saharan African Countries | 1990-2023

---

## What This Report Is About (Plain Language)

This report examines **electricity access in villages and rural areas** across Sub-Saharan Africa.

**Key Questions:**
- What percentage of village residents have electricity?
- How has rural electricity access changed over time?
- Which countries have the best/worst rural electricity access?

---

## Table of Contents
1. [Data Collection Plan](#1-data-collection)
2. [Data Loading & Cleaning](#2-data-loading)
3. [Summary Statistics](#3-statistics)
4. [Temporal Analysis](#4-temporal)
5. [Country Comparison](#5-countries)
6. [Key Findings](#6-findings)

---
# 1. Data Collection Plan <a id='1-data-collection'></a>

| Attribute | Details |
|-----------|----------|
| **Source** | World Bank - World Development Indicators |
| **Indicator** | EG.ELC.ACCS.RU.ZS |
| **Description** | Access to electricity, rural (% of rural population) |
| **Coverage** | 50 Sub-Saharan African countries |
| **Time Period** | 1990-2023 |
| **License** | CC BY-4.0 |

---
# 2. Data Loading & Cleaning <a id='2-data-loading'></a>

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

print("Libraries loaded successfully!")

In [ ]:
# Define SSA countries
ssa_countries = [
    "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi",
    "Cabo Verde", "Cameroon", "Central African Republic",
    "Chad", "Comoros", "Congo", "Côte d'Ivoire",
    "Democratic Republic of the Congo", "Djibouti",
    "Equatorial Guinea", "Eritrea", "Eswatini", "Ethiopia",
    "Gabon", "Gambia", "Ghana", "Guinea", "Guinea-Bissau",
    "Kenya", "Lesotho", "Liberia", "Madagascar", "Malawi",
    "Mali", "Mauritania", "Mauritius", "Mozambique",
    "Namibia", "Niger", "Nigeria", "Rwanda", "Sao Tome and Principe",
    "Senegal", "Seychelles", "Sierra Leone", "Somalia", "South Africa",
    "South Sudan", "Sudan", "Togo", "Tanzania",
    "Uganda", "Zambia", "Zimbabwe"
]

# World Bank to SSA name mapping
wb_to_ssa = {
    "Congo, Dem. Rep.": "Democratic Republic of the Congo",
    "Congo, Rep.": "Congo",
    "Cote d'Ivoire": "Côte d'Ivoire",
    "Gambia, The": "Gambia",
    "Tanzania": "Tanzania"
}

print(f"Target SSA countries: {len(ssa_countries)}")

In [ ]:
# Load Rural Electricity Data
df_raw = pd.read_csv('../csv/API_EG.ELC.ACCS.RU.ZS_DS2_en_csv_v2_6060.csv', skiprows=4)
print(f"Raw Data Shape: {df_raw.shape}")
print(f"\nColumns: {df_raw.columns[:10].tolist()}...")

In [ ]:
# Process data to long format
year_cols = [str(y) for y in range(1990, 2024)]
available_years = [y for y in year_cols if y in df_raw.columns]

# Select and melt
df_long = df_raw[['Country Name', 'Country Code'] + available_years].melt(
    id_vars=['Country Name', 'Country Code'],
    value_vars=available_years,
    var_name='Year',
    value_name='Rural_Electricity'
)
df_long['Year'] = df_long['Year'].astype(int)

# Standardize country names and filter to SSA
df_long['Country Name'] = df_long['Country Name'].replace(wb_to_ssa)
df_ssa = df_long[df_long['Country Name'].isin(ssa_countries)].copy()
df_ssa = df_ssa.rename(columns={'Country Name': 'Country'})

print(f"SSA Data Shape: {df_ssa.shape}")
print(f"Countries: {df_ssa['Country'].nunique()}")
print(f"Years: {df_ssa['Year'].min()} - {df_ssa['Year'].max()}")

In [ ]:
# Data quality check
print("=" * 50)
print("DATA QUALITY ASSESSMENT")
print("=" * 50)

total_rows = len(df_ssa)
missing = df_ssa['Rural_Electricity'].isna().sum()
print(f"\nTotal data points: {total_rows}")
print(f"Missing values: {missing} ({missing/total_rows*100:.1f}%)")
print(f"Valid values: {total_rows - missing}")

# Missing by country
missing_by_country = df_ssa.groupby('Country')['Rural_Electricity'].apply(lambda x: x.isna().sum())
print(f"\nCountries with most missing data:")
print(missing_by_country.sort_values(ascending=False).head(10))

In [ ]:
# Export clean dataset
df_ssa.to_csv('../csv/ssa_rural_electricity_clean.csv', index=False)
print("Clean dataset saved: ssa_rural_electricity_clean.csv")

---
# 3. Summary Statistics <a id='3-statistics'></a>

### What Are Summary Statistics? (Plain Language)

Summary statistics give us a quick overview of the data:
- **Mean (Average)**: Add up all values and divide by how many there are
- **Median**: The middle value when sorted (half above, half below)
- **Std (Standard Deviation)**: How spread out the values are from the average
- **Min/Max**: The smallest and largest values

In [ ]:
# Overall statistics
print("=" * 50)
print("RURAL ELECTRICITY ACCESS - SUMMARY STATISTICS")
print("=" * 50)

df_valid = df_ssa.dropna(subset=['Rural_Electricity'])
print("\n--- All Years Combined ---")
print(df_valid['Rural_Electricity'].describe())

In [ ]:
# Statistics by decade
df_valid['Decade'] = (df_valid['Year'] // 10) * 10
decade_stats = df_valid.groupby('Decade')['Rural_Electricity'].agg(['mean', 'median', 'std', 'min', 'max', 'count'])

print("\n--- Statistics by Decade ---")
print(decade_stats.round(1))

In [ ]:
# Distribution visualization
latest_year = df_valid['Year'].max()
df_latest = df_valid[df_valid['Year'] == latest_year]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1 = axes[0]
ax1.hist(df_latest['Rural_Electricity'].dropna(), bins=15, color='#2ca02c', edgecolor='white', alpha=0.8)
ax1.axvline(df_latest['Rural_Electricity'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df_latest["Rural_Electricity"].mean():.1f}%')
ax1.axvline(df_latest['Rural_Electricity'].median(), color='orange', linestyle='--', linewidth=2, label=f'Median: {df_latest["Rural_Electricity"].median():.1f}%')
ax1.set_xlabel('Rural Electricity Access (%)')
ax1.set_ylabel('Number of Countries')
ax1.set_title(f'Distribution of Rural Electricity Access ({latest_year})', fontweight='bold')
ax1.legend()

# Box plot by decade
ax2 = axes[1]
decade_data = [df_valid[df_valid['Decade'] == d]['Rural_Electricity'].dropna() for d in sorted(df_valid['Decade'].unique())]
bp = ax2.boxplot(decade_data, labels=[f"{int(d)}s" for d in sorted(df_valid['Decade'].unique())], patch_artist=True)
for patch in bp['boxes']:
    patch.set_facecolor('#2ca02c')
    patch.set_alpha(0.7)
ax2.set_xlabel('Decade')
ax2.set_ylabel('Rural Electricity Access (%)')
ax2.set_title('Rural Electricity Access Distribution by Decade', fontweight='bold')

plt.tight_layout()
plt.savefig('../csv/rural_elec_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

### What These Charts Show (Plain Language)

**Left Chart - Histogram:**
- Each bar shows how many countries have rural electricity access in that range
- The red dashed line is the average
- The orange dashed line is the median (middle value)
- Many countries are clustered at the low end - rural electricity access is still low in many places

**Right Chart - Box Plots by Decade:**
- Each box shows the distribution for that decade
- If boxes move upward over time, rural electricity access is improving

---
# 4. Temporal Analysis <a id='4-temporal'></a>

### What is Temporal Analysis? (Plain Language)

Temporal analysis looks at how things change **over time**. We want to see:
- Is rural electricity access increasing or decreasing?
- How fast is it changing?

In [ ]:
# SSA average over time
yearly_avg = df_valid.groupby('Year')['Rural_Electricity'].agg(['mean', 'median', 'std', 'count']).reset_index()

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(yearly_avg['Year'], yearly_avg['mean'], marker='o', linewidth=2, color='#2ca02c', label='Mean')
ax.fill_between(yearly_avg['Year'], 
                yearly_avg['mean'] - yearly_avg['std'], 
                yearly_avg['mean'] + yearly_avg['std'], 
                alpha=0.2, color='#2ca02c', label='±1 Std Dev')

ax.set_xlabel('Year')
ax.set_ylabel('Rural Electricity Access (%)')
ax.set_title('SSA Average Rural Electricity Access Over Time', fontweight='bold')
ax.legend()
ax.set_ylim(0, 100)
ax.grid(True, alpha=0.3)

# Add annotations
first_year = yearly_avg['Year'].min()
last_year = yearly_avg['Year'].max()
first_val = yearly_avg[yearly_avg['Year'] == first_year]['mean'].values[0]
last_val = yearly_avg[yearly_avg['Year'] == last_year]['mean'].values[0]

ax.annotate(f'{first_val:.1f}%', (first_year, first_val), textcoords="offset points", xytext=(0,10), ha='center', fontweight='bold')
ax.annotate(f'{last_val:.1f}%', (last_year, last_val), textcoords="offset points", xytext=(0,10), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../csv/rural_elec_trend.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nChange from {first_year} to {last_year}: {first_val:.1f}% → {last_val:.1f}% (+{last_val - first_val:.1f} percentage points)")

### What This Chart Shows (Plain Language)

- The **green line** shows the average rural electricity access across all SSA countries each year
- The **shaded area** shows how much countries vary (wider = more variation between countries)
- An **upward trend** means villages are getting better electricity access over time
- Rural access is typically much lower than urban access

In [ ]:
# Heatmap: Countries x Years
pivot_data = df_ssa.pivot(index='Country', columns='Year', values='Rural_Electricity')
pivot_data = pivot_data.sort_values(by=latest_year, ascending=False)

fig, ax = plt.subplots(figsize=(16, 14))
sns.heatmap(pivot_data, cmap='RdYlGn', center=50, linewidths=0.1, 
            cbar_kws={'label': 'Rural Electricity Access (%)'}, ax=ax)
ax.set_title('Rural Electricity Access: Countries × Years', fontweight='bold')
ax.set_xlabel('Year')
ax.set_ylabel('Country')

plt.tight_layout()
plt.savefig('../csv/rural_elec_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

### What This Heatmap Shows (Plain Language)

- Each **row** is a country, each **column** is a year
- **Green** = high electricity access (good)
- **Yellow** = medium access
- **Red** = low access (poor) - this is common for rural areas
- **White/blank** = no data available
- Countries at the **top** have the best rural electricity access

---
# 5. Country Comparison <a id='5-countries'></a>

In [ ]:
# Country ranking for latest year
country_latest = df_latest.groupby('Country')['Rural_Electricity'].mean().sort_values(ascending=True)

# Color by access level
colors = ['#d62728' if v < 20 else '#ff7f0e' if v < 50 else '#2ca02c' if v < 80 else '#1f77b4' for v in country_latest.values]

fig, ax = plt.subplots(figsize=(12, 14))
bars = ax.barh(country_latest.index, country_latest.values, color=colors)
ax.set_xlabel('Rural Electricity Access (%)')
ax.set_title(f'Rural Electricity Access by Country ({latest_year})', fontweight='bold')
ax.set_xlim(0, 105)

# Add value labels
for i, (country, val) in enumerate(country_latest.items()):
    ax.text(val + 1, i, f'{val:.1f}%', va='center', fontsize=8)

# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#1f77b4', label='>80% (Excellent)'),
    Patch(facecolor='#2ca02c', label='50-80% (Good)'),
    Patch(facecolor='#ff7f0e', label='20-50% (Moderate)'),
    Patch(facecolor='#d62728', label='<20% (Low)')
]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.savefig('../csv/rural_elec_country_ranking.png', dpi=150, bbox_inches='tight')
plt.show()

### What This Chart Shows (Plain Language)

- Each bar represents one country
- Longer bars = higher rural electricity access
- **Blue bars**: Excellent (>80%)
- **Green bars**: Good (50-80%)
- **Orange bars**: Moderate (20-50%)
- **Red bars**: Low (<20%) - many countries fall here for rural access

In [ ]:
# Countries with most improvement
first_year_data = df_valid[df_valid['Year'] == df_valid['Year'].min()].set_index('Country')['Rural_Electricity']
last_year_data = df_valid[df_valid['Year'] == df_valid['Year'].max()].set_index('Country')['Rural_Electricity']

change = (last_year_data - first_year_data).dropna().sort_values(ascending=False)

fig, axes = plt.subplots(1, 2, figsize=(14, 8))

# Top improvers
ax1 = axes[0]
top_10 = change.head(10)
colors1 = ['#2ca02c' if v > 0 else '#d62728' for v in top_10.values]
ax1.barh(top_10.index, top_10.values, color=colors1)
ax1.set_xlabel('Change (percentage points)')
ax1.set_title('Top 10 Countries: Largest Improvement', fontweight='bold')
for i, (country, val) in enumerate(top_10.items()):
    ax1.text(val + 0.5, i, f'+{val:.1f}pp', va='center', fontsize=9)

# Least improvement
ax2 = axes[1]
bottom_10 = change.tail(10)
colors2 = ['#2ca02c' if v > 0 else '#d62728' for v in bottom_10.values]
ax2.barh(bottom_10.index, bottom_10.values, color=colors2)
ax2.set_xlabel('Change (percentage points)')
ax2.set_title('Bottom 10 Countries: Least Improvement', fontweight='bold')

plt.tight_layout()
plt.savefig('../csv/rural_elec_improvement.png', dpi=150, bbox_inches='tight')
plt.show()

### What This Chart Shows (Plain Language)

- **Left chart**: Countries that improved the most in rural electricity access
- **Right chart**: Countries that improved the least
- Green bars = improvement, Red bars = decline
- Rural electrification has been a major focus in some countries

---
# 6. Key Findings <a id='6-findings'></a>

In [ ]:
# Compile key findings
print("=" * 60)
print("KEY FINDINGS: RURAL ELECTRICITY ACCESS IN SSA")
print("=" * 60)

print(f"\n1. CURRENT STATE ({latest_year})")
print(f"   - SSA Average: {df_latest['Rural_Electricity'].mean():.1f}%")
print(f"   - Median: {df_latest['Rural_Electricity'].median():.1f}%")
print(f"   - Range: {df_latest['Rural_Electricity'].min():.1f}% - {df_latest['Rural_Electricity'].max():.1f}%")

print(f"\n2. HISTORICAL CHANGE")
print(f"   - Start ({first_year}): {first_val:.1f}%")
print(f"   - End ({last_year}): {last_val:.1f}%")
print(f"   - Total change: +{last_val - first_val:.1f} percentage points")

print(f"\n3. COUNTRY RANKINGS ({latest_year})")
print(f"   - Countries with >80% rural access: {(df_latest['Rural_Electricity'] > 80).sum()}")
print(f"   - Countries with <20% rural access: {(df_latest['Rural_Electricity'] < 20).sum()}")

print(f"\n4. TOP PERFORMERS")
top_5 = country_latest.tail(5)
for country, val in top_5.items():
    print(f"   - {country}: {val:.1f}%")

print(f"\n5. DATA QUALITY")
print(f"   - Total data points: {len(df_ssa)}")
print(f"   - Missing values: {df_ssa['Rural_Electricity'].isna().sum()} ({df_ssa['Rural_Electricity'].isna().sum()/len(df_ssa)*100:.1f}%)")

## Summary (Plain Language)

### What We Learned

1. **Rural electricity access remains low**: Villages in Sub-Saharan Africa have significantly lower electricity access than cities.

2. **Access is improving but slowly**: Over the study period, rural electricity access has increased, but the average is still relatively low.

3. **Wide variation between countries**: Some countries have made significant progress in rural electrification, while others still have very low access.

4. **Many countries still below 50%**: A significant number of SSA countries still have less than half their rural population with electricity access.

### Why Rural Electricity Matters

- Without electricity, villages cannot power water pumps
- Health clinics cannot refrigerate vaccines
- Schools cannot use computers or lights for evening classes
- Economic opportunities are limited

### Data Limitations

- Some countries have missing data
- National averages may hide regional differences
- Data shows percentages only, not quality of electricity service

---

*Data Source: World Bank WDI (EG.ELC.ACCS.RU.ZS)*